In [1]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf 
import warnings
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense, Flatten,Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
import cv2
from tensorflow.keras.optimizers import Adam

In [2]:
data_path = '/kaggle/input/cell-images-for-detecting-malaria/cell_images/'
(os.listdir('/kaggle/input/cell-images-for-detecting-malaria/cell_images/'))
y=[]
for i in range(0,2):
    y.append(os.listdir('/kaggle/input/cell-images-for-detecting-malaria/cell_images/')[i])
print(y)


['Uninfected', 'Parasitized']


In [3]:
dimension_1 = []
dimension_2 = []
base_path = '/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized/'


for image_name in os.listdir(base_path):
    img_path = os.path.join(base_path, image_name)
    
   
    img = cv2.imread(img_path)
    
    if img is not None:
        # Get the dimensions of the image
        d1, d2, d3 = img.shape
        dimension_1.append(d1)
        dimension_2.append(d2)
    else:
        print(f"Error loading image: {img_path}")


Error loading image: /kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized/Thumbs.db


In [4]:
d1 = np.mean(dimension_1)
d2 = np.mean(dimension_2)
img_shape = (134,134,3)


In [5]:
base_path = '/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized/'
cell = cv2.imread(os.path.join(base_path,'C99P60ThinF_IMG_20150918_141001_cell_99.png'))
cell.max()

230

In [6]:
imageGen = ImageDataGenerator(rotation_range = 15,
                              width_shift_range = 0.08,
                              height_shift_range = 0.08,
                              shear_range = 0.08,
                              zoom_range = 0.1,
                              horizontal_flip = True,
                              fill_mode = 'nearest',
                              rescale = 1/255
)

In [7]:
imageGen.flow_from_directory('/kaggle/input/cell-images-for-detecting-malaria/cell_images/')
print(os.listdir("/kaggle/input/cell-images-for-detecting-malaria/cell_images/"))
class_indices = {'Uninfected': 0, 'Parasitized': 1, 'cell_images': 2}  # Adjust these names as needed

# Step 2: Create a custom function to filter out the 3rd class
def filter_classes(label):
    return label if label < 2 else None 

train_generator = imageGen.flow_from_directory(
    '/kaggle/input/cell-images-for-detecting-malaria/cell_images/',
    target_size=(134, 134),
    color_mode = 'rgb',
    batch_size=32,
    shuffle=True,
    classes=list(class_indices.keys())[:2],  # Only include the first two classes
    class_mode='binary'
)

Found 55116 images belonging to 3 classes.
['Uninfected', 'Parasitized', 'cell_images']
Found 27558 images belonging to 2 classes.


In [8]:
data, labels = [], []
for i in range(len(train_generator)):
    batch = next(train_generator)
    data.extend(batch[0])
    labels.extend(batch[1])
    if len(data) >= train_generator.samples:
        break

data = np.array(data)
labels = np.array(labels)

X_train_val, X_test, y_train_val, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

print(f"Number of training samples: {len(X_train)}")
print(f"Number of validation samples: {len(X_val)}")
print(f"Number of test samples: {len(X_test)}")

In [12]:
train_generator = imageGen.flow(X_train, y_train, batch_size=32)
val_generator = ImageDataGenerator(rescale=1/255).flow(X_val, y_val, batch_size=32)
test_generator = ImageDataGenerator(rescale=1/255).flow(X_test, y_test, batch_size=32)

In [15]:
model = Sequential([
    # First Convolutional Block
    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(134,134,3), padding='same'),
    BatchNormalization(),
    Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    # Second Convolutional Block
    Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    # Third Convolutional Block
    Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    # Fully Connected Layers
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 134, 134, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 134, 134, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 134, 134, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 134, 134, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 67, 67, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 67, 67, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 67, 67, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 67, 67, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 67, 67, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 67, 67, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 33, 33, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 33, 33, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 33, 33, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 33, 33, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 33, 33, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 33, 33, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    16,777,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 17,069,089 (65.11 MB)

 Trainable params: 17,067,169 (65.11 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss',patience=0,restore_best_weights=True)
checkpoint = ModelCheckpoint(
    'test_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    validation_data=val_generator,
    validation_steps=len(X_val) // 32,
    epochs=50,
    callbacks=[early_stop]
)

loss, accuracy = model.evaluate(test_generator, verbose=2)

print("Training Finished")

Epoch 1/50
697/697 ━━━━━━━━━━━━━━━━━━━━ 120s 151ms/step - accuracy: 0.6346 - loss: 0.7826 - val_accuracy: 0.7078 - val_loss: 0.8216
Epoch 2/50
697/697 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.9062 - loss: 0.4340 - val_accuracy: 0.7059 - val_loss: 0.8702
87/87 - 2s - 19ms/step - accuracy: 0.7072 - loss: 0.8370
Training Finished


In [18]:
print(accuracy)

0.7071843147277832


In [19]:
model.save('final_model_1.keras')
